In [ ]:
pip install mysql-connector-python

In [4]:
import pandas as pd
import numpy as np
import importlib.metadata
import mysql.connector
import streamlit as st

from datetime import datetime

In [6]:
# cargar el csv de prueba
df_coches = pd.read_csv("../bin/coche_prueba.csv")
df_prueba = pd.read_csv("../bin/prueba.csv")
df_prueba

,id_marca,nombre_marca
0,1,Toyota
1,2,Honda
2,3,Ford


In [8]:
# Conectar a la base de datos usando los valores almacenados en `secrets.toml`
def conectar_base_datos():
    conn = mysql.connector.connect(
        host=st.secrets["database"]["host"],
        user=st.secrets["database"]["user"],
        password=st.secrets["database"]["password"],
        database=st.secrets["database"]["database"]
    )
    return conn

In [ ]:
# Funcion para insertar coches en la base de datos
def insertar_datos(df, nombre_tabla):
    # Conexión a la base df_cochedatos de MySQL
    conn = conectar_base_datos()
    # Crear un cursor para ejecutar comandos SQL
    cursor = conn.cursor()
    # Obtener las columnas del DataFrame
    columnas = df.columns.tolist()
    # Convertir cada fila del DataFrame a una tupla
    valores = [tuple(row) for row in df.to_numpy()]

    # Crear la consulta SQL para insertar datos
    query = f"""
        INSERT INTO {nombre_tabla} ({', '.join(columnas)})
        VALUES ({', '.join(['%s'] * len(columnas))})
    """

    # Ejecutar la consulta para todas las filas
    cursor.executemany(query, valores)

    # Confirmar los cambios en la base de datos
    conn.commit()

    # Cerrar el cursor y la conexion
    cursor.close()
    conn.close()

    # Mensaje de exito
    print("Datos insertados correctamente")

# Llamar a la funcion para insertar los datos en la tabla
insertar_datos(df_prueba, 'marcas')

In [6]:
# Funcion para extraer datos
def extraer_datos(nombre_tabla):
    # Conexión a la base de datos de MySQL
    conn = conectar_base_datos()
    # Crear un cursor para ejecutar comandos SQL
    cursor = conn.cursor()

    # Crear la consulta SQL para seleccionar todos los datos de la tabla
    query = f"SELECT * FROM {nombre_tabla}"

    # Ejecutar la consulta y recuperar los datos
    cursor.execute(query)
    resultado = cursor.fetchall()

    # Obtener los nombres de las columnas de la tabla
    columnas = [desc[0] for desc in cursor.description]

    # Convertir los resultados a un DataFrame
    df = pd.DataFrame(resultado, columns=columnas)

    # Cerrar el cursor y la conexión
    cursor.close()
    conn.close()

    # Devolver el DataFrame
    return df

# Llamar a la función para extraer datos de la tabla
df_extraido = extraer_datos('marcas')
print(df_extraido)

   id_marca nombre_marca
0         1       Toyota
1         2        Honda
2         3         Ford
